In [91]:
import numpy as np
import my_lib 
import graph_lib
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.SecretSharing import Shamir
from binascii import hexlify

In [120]:
codewords = np.loadtxt("clique_of_9_words.txt", dtype=np.int)
print(codewords)

[[1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0]
 [1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 1]
 [0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 1]
 [1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1]
 [0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 0 0 1 0]]


In [92]:
def encode(filename="Van_Gohg.jpg"):
    file = open(filename, "rb")
    plainimage = file.read()
    key = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_EAX)
    #nonce = cipher.nonce
    cipherimage, tag = cipher.encrypt_and_digest(plainimage)
    file.close()
    return (key, cipherimage)

In [93]:
def split_key(key):
    shares = Shamir.split(8, 24, key)
    return shares

In [94]:
def combine_key(shares):
    key = Shamir.combine(shares)
    return key

In [115]:
def encode_shares(shares):
    encoded_shares = []
    keys_for_shares = []
    for i in np.arange(0, len(shares)):
        key = get_random_bytes(16)
        cipher = AES.new(key, AES.MODE_EAX)
        #nonce = cipher.nonce
        ciphershare, tag = cipher.encrypt_and_digest(shares[i][1])
        encoded_shares.append((shares[i][0], ciphershare))
        keys_for_shares.append((i+1, key))
    return (encoded_shares, keys_for_shares)

In [125]:
def get_family_of_sets(codewords, keys_for_shares):
    family_of_sets = []
    for i in np.arange(0, codewords.shape[0], 1):
        set = []
        for j in np.arange(0, codewords.shape[1], 1):
            if(codewords[i][j]==1):
                set.append(keys_for_shares[j])
        family_of_sets.append(set)
    return family_of_sets

In [ ]:
def decode_shares(encoded_shares, keys_for_some_shares):
    shares = []
    for i in np.arange(0, len(keys_for_some_shares), 1):
        key = keys_for_shares[i][1]
        encoded_share = 0
        for j in np.arange(0, len(encoded_shares), 1):
            if(j+1 == keys_for_some_shares[i][0]):
                encoded_share = encoded_shares[j][1]
        
        cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
        decoded_share = cipher.decrypt(encoded_share)
        try:
            cipher.verify(tag)
            print("The message is authentic:", plaintext)
        except ValueError:
            print("Key incorrect or message corrupted")
        shares.append((keys_for_some_shares[i][0], decoded_share))

In [126]:
(key, cipherimage) = encode()
print(key)

b'\xa4\xdb\x9cW\x87\xbaI0\xa5\xab\x8a\x9dm\x0bfi'


In [127]:
shares = split_key(key)
print(shares)

[(1, b'\x91J\xeb\x88.\xc3\xd7\x91\x00\xabYP\xb3=\xfb4'), (2, b'!N\xbc\xc8\xff"y\x82\xd9;[T+\'\xa6\xd5'), (3, b'\xa6W\xc43\x1f\n\x12m0\x89\xbb+\x9c\xecSu'), (4, b'\xa3\xeb\x08\xa6\xa5\x17[W\x8d\x1eM\x9b\x11}o~'), (5, b'\xec\x0b\xd7\xd0\x9c^\x90\xf0rE\xc7!Q\xe6\xfa\x88'), (6, b'z\xb52\xda\x82\tV\xec;\x84\x877\xd5\xf3\x9f '), (7, b'+\xc1\x85\xd3\xb9*\x10k;\x97\x11\x00\xc0/\x9c\x80'), (8, b'asCD\x90Q\xf2\xee\x1d\xda\x03\x07L\xa7\x05\x0e'), (9, b'\x0e\n\x00\x18\xa7\xb4y\xa3\xf6\x82)\xba\xb2AX\xa5'), (10, b'C\x85\x99\xe8\x87{\x0e\xed\x15\x1dTI\xd5|\xd5\x07'), (11, b'\xb7\xfb\xf3\xff\xf0\xe3\xe7b%I\\7\x02V\t\xa4'), (12, b'\xe6b\xe0L\x03\xff\xc4\x14\xe2$\xc5\xe8\xdd\x95r*'), (13, b'\x17\x1c\x0c\xd8T@\xcfPM8\xe7\xf3\xe1\xe9\xb0\xf3'), (14, b'\x96l\x8d\xa6\xd6\xef2G\xdd\xfe\x16]\x9b\x93\xed\xe3'), (15, b'P\t/"\x8a\x16#\xaf\x1a\x149\xba\xb2\x99P\x99'), (16, b':o\xc1\xef{\x859\xf5\x95\x1c#\xc3\x8cV\x87\xd8'), (17, b'\xed\x05\x9f\x9f\xf03\xect\xcb*\xe2\xea\x1e\xdej\x85'), (18, b'\x9d\xfc\xc6/\x8e\x

In [128]:
(encoded_shares, keys_for_shares) = encode_shares(shares)
print(encoded_shares)
print()
print(keys_for_shares)

[(1, b'x\xd1#\x90\x08\xc7\xa5\x14\xb8\xb3\x0eQl2F\x96'), (2, b'f\x07,s\xe5\xa1\xd2\x9d>\x8c`|p\xd0\xa7\x85'), (3, b'sk\x17\x85R\x0f\x1e\xfa3\xf6(Lun\xcdq'), (4, b'w\xabg\xbee\x8cB\x16\xe7\x84w\r:\x1e[o'), (5, b'\xbbe\xd7/_\x9c\xed9\xae[\x93\xa0[\x08\x8e\x0c'), (6, b'\xaa0]C\xcfA\xae\xcfO\x18\xb3\x01\t\xd6\x9f\x08'), (7, b'J\xe5\xf1\x92P\xfc\xbe2\xe0\xa1\x0c\xb7\xe4\xb9\xc5\xa1'), (8, b'\x9aU\x99.\xf4\xf8\x17X\xa3P\xadA\xdd\xc0\xb9\x8e'), (9, b'\xf2\x19\xe9\xa6Y\xc3\xcdFh\xc2\xbe\r\xd5Lv\xb2'), (10, b'\x1d\x16\x9d\xe8\xbf1\x94[\x9cdh\xa43\xae[\x02'), (11, b'1\xdd\x9b#Whb\xe2G\xfcr"~\x1c\xed\xb8'), (12, b'\xb32\xaa\xbds\x94\x1e\x14V\xfe\xf1\xeb\xf1Oi\xb6'), (13, b'L\x88\xbc\xd5\xa5\x05\xd5k\xd3\xbaZ_\x90x\x18\x19'), (14, b'\xc0i|Hz\xb7\xb7\x0cb\xc2\x8b\x8d\x81\xd4\x92\xef'), (15, b'\x03\x81\xbd\xa3\xbf\x95\xa4\x19%\xd4 p%:{\x80'), (16, b'2]]\x05\x13\xb7\xf63\xd6]\xae\x14\x85\x1bj\xc5'), (17, b'\xf8\x81\xc0\x07\xc6\xc5{b\xd3z\xb4+9X4,'), (18, b'\xc34\x87\xdf"\xc1\xedd\xa5Q\x1d\xa7\x93c\x8

In [136]:
family_of_sets = get_family_of_sets(codewords, keys_for_shares)
print(family_of_sets)

[[(1, b'\x8d:\xb0\x87\xf0\xeb\xdd\xe8\\\x13\xa7\x88\xe9\x9f\x10\xfe'), (2, b'E\x0eS\xe9\xbe\xded\xef\x06\xbc\x16\xa5"\x10;\xd0'), (3, b'v\xd1[\x8f\xda)\xe0\xea\xbd\xed\xee3\xe20\xbb\x9e'), (4, b'd\xbbU\xb6\xac\xb3^\x15\xf8\x1c"\x10\x0c\x15\x11\xf1'), (5, b'\x07\xff\xaf\x942;\x886\x9f\xe7\x99\x93&r\x00\x81'), (8, b'd\x08\x96\xd5\x1f\x9e\x17s\x88\x92\xfaM7L\x0c\r'), (11, b'\x16\xda\x87\xa6\xb6f{\\\xae\xc7`F\xcc!>\\'), (20, b'\x1c\x97q\x91n\xf3\xd5"\xe9\xde\xa2\n\t\xc5K\xd6')], [(5, b'\x07\xff\xaf\x942;\x886\x9f\xe7\x99\x93&r\x00\x81'), (6, b'*\xd78&\x80c\x88\x0e\xa6\xd2\xcb&\xcf;\xfc_'), (8, b'd\x08\x96\xd5\x1f\x9e\x17s\x88\x92\xfaM7L\x0c\r'), (10, b'\xec!1]L\x97\xa4\x02\xa5\xedH\xd6T\xaf\xcc\x06'), (12, b'ko\xeaT\xb0\xf1\xefEd\xa0\xb3q\xe36\x9a\x9d'), (14, b'\x0c\xee\xbdf\x9a\ra;^\xd5\xfc\xf4K\xc8$}'), (21, b'\x04^\xc9\x90G\x13\xafhtI&e\xb6~\xe9)'), (22, b'\xaf\x1b\xfd\xa3\x85wG]$5TJ\xb8\xdf\xd7\x03')], [(1, b'\x8d:\xb0\x87\xf0\xeb\xdd\xe8\\\x13\xa7\x88\xe9\x9f\x10\xfe'), (2, b'E\x0eS\x